In [1]:
import cv2
import numpy as np

In [2]:
# 얼굴인식
face_cascade = cv2.CascadeClassifier('D:/javaForever/util/opencv/sources/data/haarcascades/haarcascade_frontalface_default.xml')

cap = cv2.VideoCapture('facevideo.mp4')
scaling_factor = 0.5

while True:
    ret, frame = cap.read()
    if ret==True:
        frame = cv2.resize(frame, None, fx=scaling_factor, fy=scaling_factor, interpolation=cv2.INTER_AREA)
        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

        face_rects = face_cascade.detectMultiScale(gray, 1.3, 5)
        for (x,y,w,h) in face_rects:
            cv2.rectangle(frame, (x,y), (x+w,y+h), (0,255,0), 3)

        cv2.imshow('Face Detector', frame)

        c = cv2.waitKey(1)
        if c == 27:
            break
    else:
        break
        
cap.release()
cv2.destroyAllWindows()

In [4]:
# Background Subtraction 배경 지우기

cap = cv2.VideoCapture('video.mp4')

kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (3,3))
fgbg = cv2.bgsegm.createBackgroundSubtractorMOG()

# bgsegm을 포함해야 됨.

while(1):
    ret, frame = cap.read()
    if ret==True:
        fgmask = fgbg.apply(frame)
        fgmask = cv2.morphologyEx(fgmask, cv2.MORPH_OPEN, kernel)

        cv2.imshow('frame', fgmask)
        k = cv2.waitKey(1) & 0xff

        if k == 27:
            break
    else:
        break
cap.release()
cv2.destroyAllWindows()

In [5]:
# body 인식하기

cap = cv2.VideoCapture('CHARLIE PUTH - How Long  Kyle Hanagami Choreography.mp4')
font = cv2.FONT_HERSHEY_SIMPLEX

cv2.namedWindow('Face')
face_cascade = cv2.CascadeClassifier('D:/javaForever/util/opencv/sources/data/haarcascades/haarcascade_fullbody.xml')

while(True):
    ret, frame = cap.read()
    grayframe = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    
    faces = face_cascade.detectMultiScale(grayframe, 1.8, 2, 0, (30, 30))
    
    for (x,y,w,h) in faces:
        cv2.rectangle(frame, (x,y), (x+w, y+h), (255,0,0),3,4,0)
        cv2.putText(frame, 'Detected human',(x-5,y-5), font, 0.9, (255,255,0),2)
        
    cv2.imshow('Face',frame)
    
    c = cv2.waitKey(10)
    if c == 27:
        break

cap.release()
cv2.destroyAllWindows()

In [7]:
# 가면쓰는거...내 컴퓨터에서는 되지 않아..ㅠㅠ

face_cascade = cv2.CascadeClassifier('D:/javaForever/util/opencv/sources/data/haarcascades/haarcascade_frontalface_alt.xml')

face_mask = cv2.imread('mask.jpg')
h_mask, w_mask = face_mask.shape[:2]

if face_cascade.empty():
    raise IOError('Unable to load the face cascade classifier xml file')

cap = cv2.VideoCapture('facevideo.mp4')
scaling_factor = 0.5

while True:
    ret, frame = cap.read()
    frame = cv2.resize(frame, None, fx=scaling_factor, fy=scaling_factor, interpolation=cv2.INTER_AREA)
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    face_rects = face_cascade.detectMultiScale(gray, 1.3, 5)
    for (x,y,w,h) in face_rects:
        if h > 0 and w > 0:
            x = int(x + 0.1*w)
            y = int(y + 0.4*h)
            w = int(0.8 * w)
            h = int(0.8 * h)

            frame_roi = frame[y:y+h, x:x+w] 
            face_mask_small = cv2.resize(face_mask, (w, h), interpolation=cv2.INTER_AREA)

            gray_mask = cv2.cvtColor(face_mask_small, cv2.COLOR_BGR2GRAY)
            ret, mask = cv2.threshold(gray_mask, 50, 255, cv2.THRESH_BINARY)
            mask_inv = cv2.bitwise_not(mask)
            masked_face = cv2.bitwise_and(face_mask_small, face_mask_small, mask=mask)
            masked_frame = cv2.bitwise_and(frame_roi, frame_roi, mask=mask_inv)
            frame[y:y+h, x:x+w] = cv2.add(masked_face, masked_frame)

    cv2.imshow('Face Detector', frame)

    c = cv2.waitKey(1)
    if c == 27:
        break

cap.release()
cv2.destroyAllWindows()

error: OpenCV(3.4.1) C:\bld\opencv_1520732670222\work\opencv-3.4.1\modules\core\src\arithm.cpp:241: error: (-215) (mtype == 0 || mtype == 1) && _mask.sameSize(*psrc1) in function cv::binary_op


In [13]:
import math
import cv2
import numpy as np

# 이미지에서 눈동자인식

img = cv2.imread('face.jpg')
scaling_factor = 0.7

img = cv2.resize(img, None, fx=scaling_factor, fy=scaling_factor, interpolation=cv2.INTER_AREA)
cv2.imshow('Face', img)
gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

ret, thresh_gray = cv2.threshold(gray,220,255,cv2.THRESH_BINARY)
# 윤곽선을 찾는 것
value, contours, hierarchy = cv2.findContours(thresh_gray, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)

for contour in contours:
    area = cv2.contourArea(contour)
    rect = cv2.boundingRect(contour)
    x,y,width,height=rect
    radius = 0.25*(width+height)
    
    area_condition = (100<=area<=200)
    symmetry_condition = (abs(1-float(width)/float(height))<=0.2)
    fill_condition = (abs(1-(area/(math.pi*math.pow(radius,2.0))))<=0.3)
    
    if area_condition and symmetry_condition and fill_condition:
        cv2.circle(img,(int(x+radius), int(y+radius)), int(1.3*radius),(0,180,0),-1)
        
    cv2.imshow('Pupil Detector', img)
    c=cv2.waitKey()
    cv2.destroyAllWindows()

In [14]:
# 이미지에서 눈동자인식

img = cv2.imread('face.jpg')
scaling_factor = 0.7

img = cv2.resize(img, None, fx=scaling_factor, fy=scaling_factor, interpolation=cv2.INTER_AREA)
cv2.imshow('Face', img)
gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

ret, thresh_gray = cv2.threshold(gray,220,255,cv2.THRESH_BINARY)
# 윤곽선을 찾는 것
value, contours, hierarchy = cv2.findContours(thresh_gray, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)

for contour in contours:
    area = cv2.contourArea(contour)
    rect = cv2.boundingRect(contour)
    x,y,width,height=rect
    radius = 0.25*(width+height)
    
    area_condition = (100<=area<=200)
    symmetry_condition = (abs(1-float(width)/float(height))<=0.2)
    fill_condition = (abs(1-(area/(math.pi*math.pow(radius,2.0))))<=0.3)
    
    if area_condition and symmetry_condition and fill_condition:
        cv2.circle(img,(int(x+radius), int(y+radius)), int(1.3*radius),(0,180,0),-1)
        
cv2.imshow('Pupil Detector', img)
c=cv2.waitKey(1)
cv2.destroyAllWindows()

In [15]:
# 동그라미로 얼굴 인식을 하는데 이게 더 잘 되는 것 같음

#!/opt/local/bin/python
# -*- coding: utf-8 -*-
import cv2

#재생할 파일 
VIDEO_FILE_PATH = 'facevideo.mp4'

# 동영상 파일 열기
cap = cv2.VideoCapture(VIDEO_FILE_PATH)

#잘 열렸는지 확인
if cap.isOpened() == False:
    print ('Can\'t open the video (%d)' % (VIDEO_FILE_PATH))
    exit()

titles = ['orig']
#윈도우 생성 및 사이즈 변경
for t in titles:
    cv2.namedWindow(t)

#재생할 파일의 넓이 얻기
width = cap.get(cv2.CAP_PROP_FRAME_WIDTH)
#재생할 파일의 높이 얻기
height = cap.get(cv2.CAP_PROP_FRAME_HEIGHT)
#재생할 파일의 프레임 레이트 얻기
fps = cap.get(cv2.CAP_PROP_FPS)

print('width {0}, height {1}, fps {2}'.format(width, height, fps))

#XVID가 제일 낫다고 함.
#linux 계열 DIVX, XVID, MJPG, X264, WMV1, WMV2.
#windows 계열 DIVX
#저장할 비디오 코덱
fourcc = cv2.VideoWriter_fourcc(*'DIVX')
#저장할 파일 이름
filename = 'sprite_with_face_detect.avi'

#파일 stream 생성
out = cv2.VideoWriter(filename, fourcc, fps, (int(width), int(height)))
#filename : 파일 이름
#fourcc : 코덱
#fps : 초당 프레임 수
#width : 넓이
#height : 높이

#얼굴 인식용
face_cascade = cv2.CascadeClassifier()
face_cascade.load('D:/javaForever/util/opencv/sources/data/haarcascades/haarcascade_frontalface_default.xml')


while(True):
    #파일로 부터 이미지 얻기
    ret, frame = cap.read()
    #더 이상 이미지가 없으면 종료
    #재생 다 됨
    if frame is None:
        break;

    #얼굴인식 영상 처리
    grayframe = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    blur =  cv2.GaussianBlur(grayframe,(5,5), 0)
    faces = face_cascade.detectMultiScale(blur, 1.8, 2, 0, (50, 50))

    #원본 이미지에 얼굴 인식된 부분 표시
    for (x,y,w,h) in faces:
        cx = int(x+(w/2))
        cy = int(y+(h/2))
        cr = int(w/2)
        cv2.circle(frame,(cx,cy),cr,(0,255,0),3)

    # 얼굴 인식된 이미지 화면 표시
    cv2.imshow(titles[0],frame)

    # 인식된 이미지 파일로 저장
    out.write(frame)

    #1ms 동안 키입력 대기
    if cv2.waitKey(1) == 27:
        break;


#재생 파일 종료
cap.release()
#저장 파일 종료
out.release()
#윈도우 종료
cv2.destroyAllWindows()

width 1280.0, height 720.0, fps 30.0


In [6]:
# 얼굴인식
face_cascade = cv2.CascadeClassifier('D:/javaForever/util/opencv/sources/data/haarcascades/haarcascade_frontalface_default.xml')
eye_cascade = cv2.CascadeClassifier('D:/javaForever/util/opencv/sources/data/haarcascades/haarcascade_eye.xml')

cap = cv2.VideoCapture('facevideo.mp4')
scaling_factor = 0.5

while True:
    ret, frame = cap.read()
    
    if ret==True:
        minisize = (int(frame.shape[1]/3), int(frame.shape[0]/3))
        miniframe = cv2.resize(frame, minisize)
        
        gray = cv2.cvtColor(miniframe, cv2.COLOR_BGR2GRAY)
        faces = face_cascade.detectMultiScale(gray, 1.3, 5)
        
        for (x,y,w,h) in faces:
            cv2.rectangle(frame, (x,y), (x+w,y+h), (0,255,0), 3)
            roi_gray = gray[y:y+h, x:x+w]
            roi_color = miniframe[y:y+h, x:x+w]
            
            eyes = eye_cascade.detectMultiScale(roi_gray)
            for (ex,ey,ew,eh) in eyes:
                cv2.rectangle(roi_color, (ex,ey), (ex+ew,ey+eh), (0,255,0), 3)
            
        cv2.imshow('Face Detector', miniframe)

        c = cv2.waitKey(1)
        if c == 27:
            break
    else:
        break
        
cap.release()
cv2.destroyAllWindows()

In [3]:
import cv2

face_cascade = cv2.CascadeClassifier('D:/javaForever/util/opencv/sources/data/haarcascades/haarcascade_frontalface_default.xml')
eye_cascade = cv2.CascadeClassifier('D:/javaForever/util/opencv/sources/data/haarcascades/haarcascade_eye.xml')

img = cv2.imread('face.jpg')
gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
faces = face_cascade.detectMultiScale(gray, 1.3,5)
for (x,y,w,h) in faces:
    cv2.rectangle(img, (x,y), (x+w, y+h), (255,0,0),2)
    cropped = img[y - int(h/4):y + h + int(h/4), x - int(w/4):x + w + int(w/4)]
    roi_gray = gray[y:y+h, x:x+w]
    roi_color = img[y:y+h, x:x+w]
    eyes = eye_casecade.detectMultiScale(roi_gray)
    for (ex, ey, ew, eh) in eyes:
        cv2.rectangle(roi_color, (ex,ey), (ex+ew, ey+eh),(0,255,0),2)

    cv2.imshow('Image view', cropped)
    cv2.waitKey(0)
    cv2.destroyAllWindows()